# Creating database in SQLAlchemy
***

This script described the process of creating a table in a database with SQLAlchemy based on [ORM](https://en.wikipedia.org/wiki/Object-relational_mapping) methodology.

### Install PostgreSql local server
For this lab, you must install a local PostgreSQL server. We can find instruction on the installation process:
 - [Windows](https://www.postgresqltutorial.com/postgresql-getting-started/install-postgresql/)
 - [Linux](https://www.postgresqltutorial.com/postgresql-getting-started/install-postgresql-linux/)
 - [MacOs](https://www.postgresqltutorial.com/postgresql-getting-started/install-postgresql-macos/)

### Assumptions

Suppose we want to create tables for example like this:

- users

| id 	| email    |
|:--:	|:-------: |
|id_0	|email_0   |

Based on the database schema:
![db schema](images/db_Airbnb.png)

Implement the script to create this database in the local installation:
- PostgreSQL

## Prepare the environment

To correct working of this process we need create connection to database.

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

config_PostgreSQL = {
    "database_type": "postgresql",
    "user": "",
    "password": "",
    "database_url": "localhost",
    "port": 5432,
    "database_name": ""
}

db_string = "{database_type}://{user}:{password}@{database_url}:{port}/{database_name}".format(**config_PostgreSQL)

engine = create_engine(db_string)

# test the connection
try:
    conn = engine.connect()
    print("Connected successfully!")
except:
    print("Failed to connect")

Base = declarative_base()

Connected successfully!


Nowadays when we using the orm technique we need describing the database tables using class. This process has name [Declarative System](https://docs.sqlalchemy.org/en/13/orm/extensions/declarative/index.html). In SQLAlchemy we use this functionality by API [declarative](https://docs.sqlalchemy.org/en/13/orm/extensions/declarative/api.html).

Line *Base = declarative_base()* run automatically process of mapping existing tables in a database to class and give us tools to declare new tables schema.

## Creating tables structure

To declaration new table schema we need to add tools to mapping variable types from the database to Python:

In [2]:
from sqlalchemy import Column, Integer, String, Date, ForeignKey, MetaData, SmallInteger, Text, Table, Float

Now we can describe tables:

In [3]:
metadata = MetaData()

class Users(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    email = Column(String(50), unique=True)

    def __repr__(self):
            return "<users(id='{0}', email={1})>".format(
                self.id, self.email)

To correct work *declarative_base* mechanizm we must set **\_\_tablename\_\_** attribute. Database server use **__tablename__** as table name. 
In column **email** we use *String(50)* to make constraint to string length storage in the database. 

Attribute *primary_key=True* define Primary Key of this table. Columns name used on the database server are identical like class attributes name in Python. Function *\_\_repr\_\_* is decorator to string formatting after query results.
 
This procedure is equivalent of SQL code:

```sql
CREATE TABLE public.users
(
    id integer NOT NULL DEFAULT nextval('users_id_seq'::regclass),
    email character varying(50),
    CONSTRAINT users_pkey PRIMARY KEY (id)
)

TABLESPACE pg_default;

ALTER TABLE public.users
    OWNER to postgres;
```

Let's assume that we want now create another new tables, where we create the relation between them.

In [4]:
class Bookings(Base):
    __tablename__ = 'bookings'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    place_id = Column(Integer, ForeignKey('places.id'))
    start_date = Column(Date)
    end_date = Column(Date)
    price_per_night = Column(Float)
    num_nights = Column(Integer)

class Places(Base):
    __tablename__ = 'places'
    id = Column(Integer, primary_key=True)
    host_id = Column(Integer, ForeignKey('hosts.id'), unique = True)
    address = Column(String(50))
    city_id = Column(Integer, ForeignKey('cities.id'))

class Reviews(Base):
    __tablename__ = 'reviews'
    id = Column(Integer, primary_key=True)
    booking_id = Column(Integer, ForeignKey('bookings.id'))
    rating = Column(SmallInteger)
    review_body = Column(Text)

class Hosts(Base):
    __tablename__ = 'hosts'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))

class Cities(Base):
    __tablename__ = 'cities'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    country_id = Column(Integer, ForeignKey('countries.id'))

class Countries(Base):
    __tablename__ = 'countries'
    id = Column(Integer, primary_key=True)
    country_code = Column(String(50))
    name = Column(String(50))

To create full schema of table in database we use script:

In [5]:
Base.metadata.create_all(engine)

If we want create only one table (for example *Users*) we can use formula:

```python
Users.__table__.create(engine)
```

If we want to check if a given structure already exists in the database, we can use the following script:

In [6]:
from sqlalchemy_utils import database_exists, create_database

if not database_exists(engine.url):
    print("Database does not exist. Creating database...")
    create_database(engine.url)
    print("Database created successfully.")
else:
    # Connect the database if exists.
    print("Database already exists. Connecting to the database...")
    engine.connect()
    print("Connected to the database.")
    
    print("Creating tables if they do not exist...")
    Base.metadata.create_all(engine)
    print("Tables checked/created successfully.")

Database already exists. Connecting to the database...
Connected to the database.
Creating tables if they do not exist...
Tables checked/created successfully.


It is example of basic relation. For more options and type relationship between two tables look on [link](https://docs.sqlalchemy.org/en/14/orm/basic_relationships.html).

### The alternative method of a table schema declaration

Of course, the table creation method presented in the previous section is not the only one. We can describe the table using object *Table* and create a schema of this table by function *create*.

***Example:***

For SQLAlchemy >2.0:

```python
from sqlalchemy import Table, MetaData

meta = MetaData()
meta.reflect(bind=engine)

USER = Table('users', meta,
             Column('id', Integer, primary_key = True),
             Column('email', String(50)),
             extend_existing=True)

USER.create(engine)
```